# Upper Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ x := U x $ where $ U $ is upper triangular.  Vector $ y $ is not to be touched (and, indeed, not even passed into the routines).  This is a little trickier than you might think, especially when you work with lower triangular matrix $ L $, later.  Indeed, you may want to do a few small problems by hand if you don't get the right answer.  Also, PictureFLAME may help you see what is going on.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

## The routine <br> <code> Trmv_un_unb_var1( U, x ) </code>

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := U x $.  The "_un_" in the name of the routine indicates this is the "upper, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.scal( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
import flame
import laff as laff

def Trmv_un_unb_var1(U, x):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    while UTL.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.scal(upsilon11, chi1)
        laff.dots(u12t, x2, chi1)

        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

    flame.merge_2x1(xT, \
                    xB, x)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

U = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
xold = matrix( random.rand( 4,1 ) )

# Notice that U is not upper triangular.  We will only use the upper triangular part.

print( 'U before =' )
print( U )

print( 'x before =' )
print( x )

U before =
[[0.39787513 0.38811126 0.1554446  0.62945423]
 [0.43831119 0.25074414 0.72351537 0.70863105]
 [0.55817981 0.26673516 0.46919849 0.49833077]
 [0.19034323 0.83051786 0.73862089 0.96867006]]
x before =
[[0.38844616]
 [0.83172305]
 [0.79361282]
 [0.11968123]]


In [3]:
import numpy as np
laff.copy( x, xold )   # save the original vector x

Trmv_un_unb_var1( U, x )

print( 'x after =' )
print( x )

print( np.triu( U ) * xold )

print( 'x - ( np.triu( U ) * xold ) = ' ) #np.triu makes the matrix upper triangular
print( x - ( np.triu( U ) * xold ) )

x after =
[[0.67605083]
 [0.86755059]
 [0.43200278]
 [0.11593162]]
[[0.67605083]
 [0.86755059]
 [0.43200278]
 [0.11593162]]
x - ( np.triu( U ) * xold ) = 
[[0.]
 [0.]
 [0.]
 [0.]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The routine <br> <code> Trmv_un_unb_var2( U, x ) </code>

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := U x $.  The "_un_" in the name of the routine indicates this is the "upper triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
<li> <code> laff.scal( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
def Trmv_un_unb_var2(U, x):

    UTL, UTR, \
    UBL, UBR  = flame.part_2x2(U, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    while UTL.shape[0] < U.shape[0]:

        U00,  u01,       U02,  \
        u10t, upsilon11, u12t, \
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, \
                                                         UBL, UBR, \
                                                         1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.axpy(chi1, u01, x0)
        laff.scal(upsilon11, chi1)

        #------------------------------------------------------------#

        UTL, UTR, \
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  \
                                               u10t, upsilon11, u12t, \
                                               U20,  u21,       U22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

    flame.merge_2x1(xT, \
                    xB, x)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

U = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
xold = matrix( random.rand( 4,1 ) )

# U is not upper triangular.  We will only use the upper triangular part.

print( 'U before =' )
print( U )

print( 'x before =' )
print( x )


U before =
[[0.38401273 0.34461368 0.10622314 0.08022451]
 [0.28432701 0.11739356 0.01308147 0.25126554]
 [0.77660891 0.41993476 0.176514   0.29553577]
 [0.46893241 0.87490424 0.53882804 0.5492167 ]]
x before =
[[0.25757924]
 [0.18359536]
 [0.17964237]
 [0.63464787]]


In [6]:
laff.copy( x, xold )   # save the original vector y

Trmv_un_unb_var2( U, x )

print( 'x after =' )
print( x )

print( 'x - ( np.triu( U ) * xold ) = ' ) #np.triu makes the matrix upper triangular
print( x - ( np.triu( U ) * xold ) )

x after =
[[0.23217967]
 [0.18336804]
 [0.21927054]
 [0.34855921]]
x - ( np.triu( U ) * xold ) = 
[[ 0.00000000e+00]
 [-2.77555756e-17]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.